In [1]:
import numpy as np
import ipywidgets as ipw
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()
from ipywidgets import interact
from collections import OrderedDict

# Ignore warnings about overflow data points
old_settings = np.seterr(over='ignore') 

Loading BokehJS ...

# Theoretical Background

## Electron Dispersion Relations and Landau Quantization

Electrons in crystalline solids exhibit specific dispersion relations which denote the dependence of their energy on their momentum (or wave vector). The graph of these relations in momentum space is known as the **Fermi Surface**. The Fermi surface is pivotal because it divides the occupied and unoccupied electron states at absolute zero temperature.

When a magnetic field is applied to a metallic solid, the electron orbits are quantized, resulting in a phenomenon known as **Landau quantization**. In this scenario, electrons whirl around the direction of the magnetic field in cyclotron orbits while also performing a drifting motion perpendicular to both the electric and magnetic fields. This quantization leads to the formation of Landau levels, which are quantized energy levels for the electrons in the magnetic field.

### Key Concepts and Equations:

1. **Dispersion Relation:** A mathematical description providing the energy of an electron as a function of its wave vector e.g., \$ E(\mathbf{k}) $\.
2. **Fermi Surface:** A surface in reciprocal space which separates occupied from unoccupied electron states.
3. **Landau Levels:** Discrete energy levels that arise due to electron motion in a magnetic field.
4. **Landau Quantization:** Refers to the quantization of the cyclotron orbits of charged particles in magnetic fields.

## Questions for Self-study

### Basic Understanding:

1. **Electron Dispersion:** How do electron dispersion relations depend on the crystal structure?
2. **Fermi Surface Implications:** What physical properties of a solid are dictated by the Fermi surface?
3. **Landau Level Origination:** What are Landau levels, and how are they originated?
4. **Electron Behavior:** How do electrons behave in a magnetic field, and how is this depicted in the code?
   
### Advanced Insights:

5. **Quantization Aspect:** How and why does the quantization of the orbital magnetic moment occur?
6. **Electron Gas:** How is an electron gas formulated, and what roles do Fermi energy and Fermi momentum play in it?
7. **Magnetic Field Impact:** How does a magnetic field affect electron states in metals and semiconductors?
   
### Extension and Application:

8. **Interaction with Phonons:** How can the script be extended to incorporate interactions with phonons or other particles?
9. **Other Lattice Types:** How can the script be adapted to represent electrons in different lattice types or different materials?
10. **Temperature Effect:** How does temperature affect the electronic structure and Fermi surface, and how could this be implemented in the code?


In [2]:
# Variable declarations and initializations
kz = np.linspace(-np.pi, np.pi)
Ez = -2*np.cos(kz) - 3.0
B = 0.2  
hbar = 1.0  
e = 1.0 
m = 1.0  
c = 1.0 
wc = e*B/(m*c)

# Define the Fermi surface in zero magnetic field
freeE = figure(height=400, width=400, title="Dispersion relationship of a free electron",
               tools="pan,reset,save,wheel_zoom",
               x_range=[1.1*np.amin(Ez), -1.1*np.amin(Ez)],
               y_range=[np.amin(kz), np.amax(kz)])

# DataSource for lines
source1 = ColumnDataSource(data={'xVal': -Ez, 'yVal': kz, 'xxVal': Ez})
freeE.line('xVal', 'yVal', source=source1, line_width=2, line_alpha=1)
freeE.line('xxVal', 'yVal', source=source1, line_width=2, line_alpha=1)
freeE.xaxis.axis_label = 'kx & ky'
freeE.yaxis.axis_label = 'kz'

# More calculations
dE = hbar*wc
dA = 2*m*np.pi*wc/hbar
A = np.arange(np.amin(Ez)**2*np.pi, np.amax(Ez)**2*np.pi, -dA)
kk = np.sqrt(A/np.pi)
kF = np.array([np.flip(-kk), kk]).flatten()
Ef = np.array([np.flip(-np.arccos((kk-3.0)/2.0)), np.arccos((kk-3.0)/2.0)]).flatten()
gExy = m/(np.pi*hbar**2)

# DataSource for circles
source2 = ColumnDataSource(data={'xVal': kF, 'yVal': Ef, 'yyVal': -Ef})
freeE.circle('xVal', 'yVal', source=source2, line_width=3, line_alpha=0.2)
freeE.circle('xVal', 'yyVal', source=source2, line_width=3, line_alpha=0.2)

# Histogram calculations
bWidth = (np.amax(kz)-np.amin(kz))/100.0
histoN = (2*np.amax(kz)/bWidth).astype(int)
kCount = np.empty(histoN, dtype=np.int8)
gEs = np.empty(histoN, dtype=np.double)
kIntv = np.linspace(kz.min(), kz.max(), histoN)
zero = np.zeros(histoN)

# Calculating and plotting histograms
for idx in range(histoN):
    kCount[idx] = len(Ef[(Ef >= kIntv[idx-1]) & (Ef < kIntv[idx])])
    gEs[idx] = kCount[idx]*gExy/(0.001 + np.abs(np.sin(0.5*(kIntv[idx-1] + kIntv[idx]))))

# Preparing the figures
histo, histo2 = [figure(height=400, width=300, title=title, tools="pan,reset,save,wheel_zoom",
                        y_range=[np.amin(kz), np.amax(kz)], x_range=[0, 1.1*np.amax(gEs)]) 
                 for title in ["allowed states per Energy interval", "extrema orbitals"]]

# DataSource for histograms
source3 = ColumnDataSource(data={'bottom': kIntv, 'top': kIntv + bWidth, 'right': kCount, 'left': zero})
source4 = ColumnDataSource(data={'bottom': kIntv, 'top': kIntv + bWidth, 'right': gEs, 'left': zero})

# Plotting histograms
histo.quad(bottom='bottom', top='top', right='right', left='left', source=source3, fill_color="navy", line_color="white", alpha=1)
histo2.quad(bottom='bottom', top='top', right='right', left='left', source=source4, fill_color="navy", line_color="white", alpha=1)

# Display the plots
show(row(freeE, histo, histo2))


# Haas-van Alphan effect

### Next, we use the above method to compute the density of states at different external fields to visualize the de Haas-van Alphen effect

In [3]:
# Remove variables to avoid conflicts
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [4]:
import numpy as np
import ipywidgets as ipw

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

from ipywidgets import interact
from collections import OrderedDict

# Ignore warnings about overflow data points
old_settings = np.seterr(over='ignore')

# Define a function to compute densities of states at different external magnetic fields
def LandauDen(B):
    kz = np.linspace(-np.pi, np.pi)
    Ez = -2*np.cos(kz) - 3.0
    hbar, e, m, c = 1.0, 1.0, 1.0, 1.0  # Electron properties and speed of light
    wc = e * B / (m * c)  # Cyclotron frequency
    dE = hbar * wc
    A = np.arange(np.amin(Ez)**2 * np.pi, np.amax(Ez)**2 * np.pi, -2 * m * np.pi * wc / hbar)
    kk = np.sqrt(A / np.pi)
    kF = np.array([np.flip(-kk), kk])
    kF = kF.flatten()
    # Ensure that the argument of arccos is within [-1, 1] to avoid NaN values
    Ef = np.arccos(np.clip((kk-3.0)/2.0, -1, 1))
    Ef = np.array([np.flip(-Ef), Ef])
    Ef = Ef.flatten()
    gExy = m / (np.pi * hbar**2)
    bWidth = (np.amax(kz) - np.amin(kz)) / 100.0
    histoN = (2 * np.amax(kz) / bWidth).astype(int)
    gEs = np.empty(histoN, dtype=np.double)
    haas = np.empty(histoN, dtype=np.double)
    kIntv = np.linspace(kz.min(), kz.max(), histoN)
    zero = np.zeros(histoN)
    for idx in range(histoN-1):
        temp = len(Ef[ (Ef >= kIntv[idx]) & (Ef <= kIntv[idx+1]) ])
        gEs[idx] = temp * gExy / (0.001 + np.abs(np.sin(0.5 * (kIntv[idx+1] + kIntv[idx]))))
        haas = abs(np.sin(Ef**-2))
    return np.sum(gEs)

# Generate a range of B values and corresponding density of states
invB = np.arange(0.1, 1, 0.001)
B = 1/invB
densStat = np.array([LandauDen(b) for b in B])

# Prepare the source data for Bokeh plots
source5 = ColumnDataSource(data={'xVal': B, 'yVal': densStat})
source6 = ColumnDataSource(data={'xVal': invB, 'yVal': densStat})

# Create Bokeh plots
dHvA = figure(height=350, width=450, title="Density of states of free electrons in magnetic field")
dHvA.line('xVal', 'yVal', source=source5, line_width=2, line_alpha=1)
dHvA.xaxis.axis_label = 'Magnetic Field (B)'
dHvA.yaxis.axis_label = 'Density of States'

dHvA2 = figure(height=350, width=450, title="Density of states of free electrons in magnetic field")
dHvA2.line('xVal', 'yVal', source=source6, line_width=2, line_alpha=1)
dHvA2.xaxis.axis_label = 'Inverse of Magnetic Field (1/B)'
dHvA2.yaxis.axis_label = 'Density of States'

# Display the plots
show(row(dHvA, dHvA2))


Loading BokehJS ...

# Self-Education Questions on Solid State Physics Script

## 1. Basics of Quantum Mechanics and Statistical Physics:
   - What fundamental principles of quantum mechanics underpin this script?
   - What is density of states and how is it related to this script?
   - Explain the significance of each parameter in the `LandauDen` function and how they affect the results.

## 2. Electronic Properties of Solid State Materials:
   - How does an external magnetic field affect the electronic states in a solid material?
   - What is the de Haas–van Alphen effect and how is it manifested in this script?
   - Explain the physical interpretation of the graphs generated by running this script.

## 3. Programming and Visualization:
   - Explain how the `LandauDen` function operates. What are the key steps involved in this function and how do they correlate with the physical processes?
   - What libraries and tools are utilized in this script for data visualization? What alternatives exist?
   - How could you modify the script to analyze the electronic properties of solid materials under various external conditions (e.g., varying temperature or pressure)?

## 4. Application of Acquired Knowledge:
   - What practical applications can be found for analyzing the electronic properties of solid materials in an external magnetic field?
   - What further research or experiments could be proposed based on the results obtained from this script?
